https://github.com/studioimaginaire/phue 

Phue is a package for interacting with the Hue hub. 

I have 2 remotes, 1 tap, and 3 motion (+ temp and light) sensors. Hue also creates 'sensors' for presence ('HomeAway', 'Robins iPhone') as well as dimmer secnes.

In [1]:
from phue import Bridge
from datetime import datetime, timedelta
from functools import wraps

First create dummy Throttle class stolen from https://gist.github.com/ChrisTM/5834503 as data updates will be throttled

In [2]:
class Throttle(object):
    """
    Decorator that prevents a function from being called more than once every
    time period.
    To create a function that cannot be called more than once a minute:
        @throttle(minutes=1)
        def my_fun():
            pass
    """
    def __init__(self, seconds=0, minutes=0, hours=0):
        self.throttle_period = timedelta(
            seconds=seconds, minutes=minutes, hours=hours
        )
        self.time_of_last_call = datetime.min

    def __call__(self, fn):
        @wraps(fn)
        def wrapper(*args, **kwargs):
            now = datetime.now()
            time_since_last_call = now - self.time_of_last_call

            if time_since_last_call > self.throttle_period:
                self.time_of_last_call = now
                return fn(*args, **kwargs)

        return wrapper

# Setup variables

In [3]:
bridge_ip = '192.168.0.3'
SCAN_INTERVAL = 1   # Seconds

# Sensors
Sensors are more simple than lights in some ways, as lights can be in groups. A significant part of the HA lights component is dealing with groups. 

For SML motion sensors we have the issue that Hue treates these device as 3 seperate sensors consisting of motion, temperature and light level. I prefer to group by device so need to have a routine which correctly ties sensors to a device. The dimmer remote has scenes but will ignore these.

Lets create a wrapper to the bridge object that throttles requests for sensor data.

In [30]:
class HueSensorData(object):
    """Get the latest sensor data."""
    # need to import phue
    def __init__(self, bridge_ip):
        """Initialize the data object and count devices."""
        self._bridge = Bridge(bridge_ip)
        self.update()  # Fetch data
        
    # Update only once in scan interval.
    @Throttle(seconds=SCAN_INTERVAL)
    def update(self):
        """Get the latest data but throttle requests."""
        self._data = self._bridge.get_sensor()
        
    @property
    def data(self):
        """Return the data."""
        return self._data

In [31]:
sensor_data_obj = HueSensorData(bridge_ip)

lets get a list of the sensor id available in data

In [33]:
sensor_id_list = list(sensor_data_obj.data.keys())
sensor_id_list

['1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 '10',
 '11',
 '12',
 '13',
 '14',
 '15',
 '16',
 '18',
 '20',
 '21',
 '22',
 '23',
 '24']

In [34]:
sensor_data_obj.data['10']

{'config': {'alert': 'none',
  'battery': 100,
  'ledindication': False,
  'on': True,
  'pending': [],
  'reachable': True,
  'tholddark': 12853,
  'tholdoffset': 7000,
  'usertest': False},
 'manufacturername': 'Philips',
 'modelid': 'SML001',
 'name': 'Hue ambient light sensor 2',
 'state': {'dark': True,
  'daylight': False,
  'lastupdated': '2018-01-19T06:13:48',
  'lightlevel': 0},
 'swupdate': {'lastinstall': None, 'state': 'noupdates'},
 'swversion': '6.1.0.18912',
 'type': 'ZLLLightLevel',
 'uniqueid': '00:17:88:01:02:00:b5:ce-02-0400'}

In [35]:
for sensor_id in sensor_id_list:
    obj = sensor_data_obj.data[sensor_id]
    if obj['modelid'] in ['SML001', 'RWL021', 'ZGPSWITCH', 'HOMEAWAY']:
        print("{}, {} : modelid = {}, device_id = {}".format(sensor_id, obj['name'], obj['modelid'], obj['uniqueid'].split(':')[-1][0:5]))

2, Remote bedroom : modelid = RWL021, device_id = dc-02
4, Hue temperature sensor 1 : modelid = SML001, device_id = 28-02
5, Hall Sensor : modelid = SML001, device_id = 28-02
6, Hue ambient light sensor 1 : modelid = SML001, device_id = 28-02
8, Hue temperature sensor 2 : modelid = SML001, device_id = ce-02
9, Bedroom sensor : modelid = SML001, device_id = ce-02
10, Hue ambient light sensor 2 : modelid = SML001, device_id = ce-02
13, HomeAway : modelid = HOMEAWAY, device_id = L_01_
14, Hue temperature sensor 3 : modelid = SML001, device_id = 9c-02
15, Living room sensor : modelid = SML001, device_id = 9c-02
16, Hue ambient light sensor 3 : modelid = SML001, device_id = 9c-02
20, Living room remote : modelid = RWL021, device_id = 1e-02
22, Hall remote : modelid = RWL021, device_id = 9c-02
24, Hue Tap : modelid = ZGPSWITCH, device_id = 08-f2


PHDL00 does not have a device_id

So even though we have 1 device we have different names for its temperature and light level sensors. Associate these sensors by device_id. Lets create a HueSensor object.

In [36]:
class HueSensor(object):
    """Class to hold Hue Sensor basic info."""

    def __init__(self, sensor):
        """Initialise with a phue sensor object."""
        self._device_id = sensor['uniqueid'].split(':')[-1][0:5]
        self._name = sensor['name']
        self._model = sensor['modelid']
        self._attributes = {}

In [37]:
sensor_data_obj.data['5']

{'config': {'alert': 'none',
  'battery': 100,
  'ledindication': False,
  'on': True,
  'pending': [],
  'reachable': True,
  'sensitivity': 2,
  'sensitivitymax': 2,
  'usertest': False},
 'manufacturername': 'Philips',
 'modelid': 'SML001',
 'name': 'Hall Sensor',
 'state': {'lastupdated': '2018-01-19T06:09:04', 'presence': False},
 'swupdate': {'lastinstall': None, 'state': 'noupdates'},
 'swversion': '6.1.0.18912',
 'type': 'ZLLPresence',
 'uniqueid': '00:17:88:01:02:00:af:28-02-0406'}

In [38]:
hall_sensor = HueSensor(sensor_data_obj.data['5'])

In [39]:
hall_sensor._device_id

'28-02'

Lets put the sensor objects in a list

In [28]:
entities = []
devices = {}  
for sensor_id in sensor_id_list:
    obj = sensor_data_obj.data[sensor_id]
    if obj['modelid'][0:3] in ['SML', 'RWL', 'ZGP']:
        entities.append(HueSensor(obj))
        device_id = obj['uniqueid'].split(':')[-1][0:5]
        if device_id not in devices:
            devices.append(device_id)

In [29]:
print(len(entities))
for e in entities:
    print(e._name)

14
Remote bedroom
Hue temperature sensor 1
Hall Sensor
Hue ambient light sensor 1
Hue temperature sensor 2
Bedroom sensor
Hue ambient light sensor 2
HomeAway
Hue temperature sensor 3
Living room sensor
Hue ambient light sensor 3
Living room remote
Hall remote
Hue Tap


My unique devices

In [16]:
print(len(devices))
print(devices)

7
['dc-02', '28-02', 'ce-02', 'L_01_', '9c-02', '1e-02', '08-f2']


In [ ]:
        self._devices = []  # List to keep valid devices
        for key, obj in self._data.items():
            if obj.get('modelid') in ['SML001', 'RWL021', 'ZGPSWITCH', 'HOMEAWAY']:
                self._devices.append(obj['uniqueid'].split(':')[-1][0:5])
        self._devices = set(self._devices)  # Keep only uniques